__Dhyan Thakkar__
<br>
Date: Jan. 19, 2021
<br>
PHYS 2030 W22

# <center><font color=#46769B>Exercise 4: Radioactive decay</font></center>

## <font color=#46769B>Introduction:</font>

Our goals for this notebook are:
- Make a simulation for 

Required reading:
- *Lesson 3: Sampling from arbitrary distributions*

# <font color=#46769B>Exercise</font>

## <font color=#46769B>Part (a)</font>

Consider a radioactive isotope $A$, at $t=0$. We want to know the time $t$ for one atom to decay. 
Since radioactive decay is a random process, there is no way to know in advance $t$ for any one atom. 
We will treat $t$ as a variable that we will randomly sample from a PDF $P(t)$.

The law of radioactive decay tells us that the *probability* of $A$ surviving until time $t$ is $e^{-t/\tau_A}$, where $\tau_A$ is the lifetime for its decay.
Conversely, the probability for $A$ to decay *before* time $t$ is $(1-e^{-t/\tau_A})$. The latter is nothing more than the CDF:
$$ C(t) = \left\{ \begin{array}{cc} 1 - e^{-t/\tau_A} & {\rm for} \; t \ge 0 \\
0 & {\rm otherwise} \end{array} \right.$$
Since $P(t) = dC/dt$, we now have the PDF:
$$ P(t) = \left\{ \begin{array}{cc} \frac{1}{\tau_A} e^{-t/\tau_A} & {\rm for} \; t \ge 0 \\
0 & {\rm otherwise} \end{array} \right. \, ,$$
which you can verify has the correct normalization.

Setting $\tau_A = 10$ (e.g., 10 hours), complete the following:
- Write a code `sample_t_A(num)` that will draw `num` random samples for $t$ from $P(t)$ defined above and return a `numpy.array` of length `num` with your list of times $t$.
- Determine the half-life $t_{1/2}$ from the 50\% quantile from your sample.
- Determine the time it takes for 99.9\% of $A$ to decay away by determining the 99.9\% quantile for your sample.

For the last two parts, be sure that $N$ is large enough so that you find consistent results when you perform multiple simulations.

In [6]:
import numpy as np

tau_A = 10

def sample_t_A(num):
    r = np.random.rand(num)
    t_samples = -tau_A*np.log(1-r)
    return t_samples

num = 100000
#Estimating Quantiles
print('t_50 =', np.quantile(sample_t_A(num), 0.5))
print('t_99.9 =', np.quantile(sample_t_A(num), 0.999))

t_50 = 6.881354164751353
t_99.9 = 66.78446507255836


## <font color=#46769B>Part (b)</font>

Consider a more complicated chain of decays:

<div>
<img src="https://github.com/PHYS-2030-Computational-Methods/Lecture-notes/raw/main/ABC_decay.png" width="200"/>
</div>

Isotope $A$, with lifetime $\tau_A = 10$, decays into isotope $B$ 83\% of the time and into isotope $C$ 17\% of the time.
Isotopes $B$ and $C$ are themselves radioactive, decaying with lifetimes $\tau_B = 2$ and $\tau_C=30$, respectively, into stable isotopes.

Starting with isotope $A$ at $t=0$, generate $N$ random samples for the *total* time to decay to a stable isotope. Using `numpy.quantile`, determine the following:
- Determine the total half-life $t_{1/2}$ for all radioactive isotopes to decay away.
- Determine the time it takes for 99.9\% of all radioactive isotopes to decay away.

Choose $N$ large enough so that your results are consistent for multiple simulations.

### <font color=#46769B>Hint:</font>

The total time is
$$ t_{\rm tot} = \left\{ \begin{array}{cc} t_A + t_B & {\rm 83\% \; of\; the\; time} \\ 
t_A + t_C & {\rm 17\% \; of\; the\; time} \end{array} \right. \, ,$$
where $t_A$ is the time for $A$ to decay from part (a), and $t_B$ and $t_C$ are the times for $B$ and $C$ to decay.
So, the logic is as follows:
- Randomly sample $t_A$ as in part (a).
- Randomly sample from a discrete choice: does $A \to B$ or $B \to C$?
- Randomly sample *either* $t_B$ or $t_C$ (depending on which decay occurs), as in part (a) but using the appropriate lifetime $\tau_B$ or $\tau_C$.
- Compute $t_{\rm tot} = t_A + t_B$ or $t_A + t_C$ (depending on which decay occurs).

That is __one__ sample. Repeat $N$ times do generate $N$ samples.



In [5]:
import numpy as np
#defining known variables
tauA = 10
tauB = 2
tauC = 30

def sample_t(num):
  #Random sampling A
  r = np.random.rand(num)
  tA = -tauA*np.log(1-r)
  #Define B = 1, and C = 0
  r1 = np.random.rand(num)
  tB = -tauB*np.log(1-r1)
  tC = -tauC*np.log(1-r1) 

  #Random Sampling tB and tC
  options = [tB,tC]
  prob = [0.83,0.17]
  
  samples = []
  for k in range(num):
      if r1[k] < prob[0]:
        samples.append(options[0])
      elif r1[k] < prob[0] + prob[1]:
        samples.append(options[1])
  t_total = np.add(tA,samples)
  return t_total

num = 10000
print('t_50 =', np.quantile(sample_t(num), 0.5))
print('t_99.9 =', np.quantile(sample_t(num), 0.999))

t_50 = 10.78184261942725
t_99.9 = 163.98881410381753
